In [1]:
import numpy as np
import scipy.stats as stats
from scipy.stats import poisson, norm

In [15]:
import csv

In [19]:
def expect(X, f, lb, ub):
    if hasattr(X, 'dist'):
        return X.dist.expect(f, lb = lb, ub = ub, *X.args, **X.kwds)
    else:
        return X.expect(f, lb = lb, ub = ub)

In [20]:
from scipy.integrate import quad
import pylab as pl

class newsVendor(object):
    def __init__(self, co, cs, X):
        self.co = co
        self.cs = cs
        self.X = X # demand distribution during lead time
        self.G = self.X.cdf

    def unitsOver(self, Q):
        return expect(self.X, lambda x: Q-x, 0, Q)

    def unitsShort(self, Q):
        return expect(self.X, lambda x: x-Q, Q, np.inf)

    def Y(self,Q):
        return self.co*self.unitsOver(Q) + self.cs*self.unitsShort(Q)

    def Qstar(self):
        return self.X.ppf(self.cs/(self.cs+self.co))

    def profit(self, Q):
        investedValue = self.co * Q
        expectedSales = expect(self.X, lambda x: np.minimum(x,Q), 0, np.inf)
        return (self.co + self.cs)*expectedSales - investedValue

In [12]:
price = 1.50
cost = .87

In [14]:
#this will need to be calculated from csv
mu = 10
G = poisson(mu)
p = G.pmf(np.arange(100))

In [22]:
# mu and sigma
X = norm(loc = 10000., scale = 1000.)
news = newsVendor(co = cost, cs = price, X= X)
Q =  news.Qstar()

print (Q)
print (news.Y(Q))
print (news.Y(10440)) # rounding inaccuracy in FP

10339.5741933
892.522197534361
896.9684635617612


In [23]:
norm